In [1]:
import pandas as pd
import numpy as np
import time

import json
import requests

In [2]:
def api_url2(params):
    '''
    Helper to create request string
    '''
    API_ENDPOINT = 'https://comtrade.un.org/api/get?'
    
    url = []
    for k, v in params.items():
        if isinstance(v, list):
           url.append(
               '{k}={v}'.format(k=k, v='%2C'.join(str(vi) for vi in v))
           )
        else:
            url.append('{k}={v}'.format(k=k, v=v))
    
    return API_ENDPOINT + '&'.join(url)

In [3]:
OIL_CODE = 270900

params = {
    'max': 1000,
    'type': 'C',
    'freq': 'A',
    'px': 'H1',    # HS 1996
    'ps': 2020,   # year
    'r': 842,    # USA
    'p': "all",   # Partner code separated by comma , 381, 112, 156, 804
    'rg': "all",    # 1 - Import, 2 - Export [1, 2], all
    'cc': OIL_CODE,
    'fmt': 'csv'
}

In [4]:
json_csv_cols = {
    'yr': 'Year',
    'rgDesc': 'Trade Flow',
    'rtTitle': 'Reporter',
    'rtCode': 'Reporter Code',
    'rt3ISO': 'Reporter ISO',
    'ptTitle': 'Partner',
    'ptCode': 'Partner Code',
    'pt3ISO': 'Partner ISO',
    'TradeValue': 'Trade Value (US$)',
    'cmdDescE': 'Commodity',
    'cmdCode': 'Commodity Code'
}

cols = [
    'Year',
    'Trade Flow',
    'Reporter',
    'Reporter Code',
    'Reporter ISO',
    'Partner',
    'Partner Code',
    'Partner ISO',
    'Trade Value (US$)',
    'Commodity',
    'Commodity Code',
    'Qty',
    'Qty Unit'
]

In [5]:
countries = [
    {'id': '660', 'text': 'Anguilla'}, {'id': '28', 'text': 'Antigua and Barbuda'}, {'id': '32', 'text': 'Argentina'}, {'id': '533', 'text': 'Aruba'},
    {'id': '44', 'text': 'Bahamas'}, {'id': '52', 'text': 'Barbados'}, {'id': '84', 'text': 'Belize'}, {'id': '60', 'text': 'Bermuda'},
    {'id': '68', 'text': 'Bolivia (Plurinational State of)'}, {'id': '535', 'text': 'Bonaire'}, {'id': '76', 'text': 'Brazil'},
    {'id': '92', 'text': 'Br. Virgin Isds'}, {'id': '124', 'text': 'Canada'}, {'id': '136', 'text': 'Cayman Isds'}, {'id': '152', 'text': 'Chile'},
    {'id': '170', 'text': 'Colombia'}, {'id': '188', 'text': 'Costa Rica'}, {'id': '192', 'text': 'Cuba'},
    {'id': '531', 'text': 'Curaçao'}, {'id': '212', 'text': 'Dominica'}, {'id': '214', 'text': 'Dominican Rep.'}, {'id': '218', 'text': 'Ecuador'},
    {'id': '222', 'text': 'El Salvador'}, {'id': '238', 'text': 'Falkland Isds (Malvinas)'}, {'id': '254', 'text': 'French Guiana'}, {'id': '304', 'text': 'Greenland'},
    {'id': '308', 'text': 'Grenada'}, {'id': '312', 'text': 'Guadeloupe'}, {'id': '320', 'text': 'Guatemala'}, {'id': '328', 'text': 'Guyana'},
    {'id': '332', 'text': 'Haiti'}, {'id': '340', 'text': 'Honduras'}, {'id': '388', 'text': 'Jamaica'}, {'id': '474', 'text': 'Martinique'},
    {'id': '484', 'text': 'Mexico'}, {'id': '500', 'text': 'Montserrat'}, {'id': '558', 'text': 'Nicaragua'}, {'id': '591', 'text': 'Panama'},
    {'id': '600', 'text': 'Paraguay'}, {'id': '604', 'text': 'Peru'}, {'id': '461', 'text': 'Sabah'}, {'id': '652', 'text': 'Saint Barthelemy'},
    {'id': '659', 'text': 'Saint Kitts and Nevis'}, {'id': '658', 'text': 'Saint Kitts, Nevis and Anguilla'}, {'id': '662', 'text': 'Saint Lucia'},
    {'id': '534', 'text': 'Saint Maarten'}, {'id': '666', 'text': 'Saint Pierre and Miquelon'}, {'id': '670', 'text': 'Saint Vincent and the Grenadines'},
    {'id': '740', 'text': 'Suriname'}, {'id': '780', 'text': 'Trinidad and Tobago'}, {'id': '796', 'text': 'Turks and Caicos Isds'},
    {'id': '858', 'text': 'Uruguay'}, {'id': '850', 'text': 'US Virgin Isds'}, {'id': '842', 'text': 'USA'}, {'id': '841', 'text': 'USA (before 1981)'},
    {'id': '862', 'text': 'Venezuela'},
]

In [43]:
len(countries)

56

In [4]:
api_url2(params)

'https://comtrade.un.org/api/get?max=5000&type=C&freq=A&px=HS&ps=2020&r=842&p=all&rg=1%2C2&cc=270900&fmt=csv'

In [6]:
reporter_codes = [ c['id'] for c in countries]

reporters_chunks = []
for i in range(0, len(reporter_codes), 5):
    reporters_chunks.append(reporter_codes[i:i+5])
    
years_chunks = []
years = list(range(2013, 2022))
for i in range(0, len(years), 5):
    years_chunks.append(years[i:i+5])

In [8]:
years_chunks

[[2013, 2014, 2015, 2016, 2017], [2018, 2019, 2020, 2021]]

In [7]:
reporters_chunks

[['660', '28', '32', '533', '44'],
 ['52', '84', '60', '68', '535'],
 ['76', '92', '124', '136', '152'],
 ['170', '188', '192', '531', '212'],
 ['214', '218', '222', '238', '254'],
 ['304', '308', '312', '320', '328'],
 ['332', '340', '388', '474', '484'],
 ['500', '558', '591', '600', '604'],
 ['461', '652', '659', '658', '662'],
 ['534', '666', '670', '740', '780'],
 ['796', '858', '850', '842', '841'],
 ['862']]

In [15]:
params['cc'] = OIL_CODE

for years in years_chunks:
    data = []
    params['ps'] = years
    
    for reporters in reporters_chunks:
        params['r'] = reporters
        
        print(api_url2(params))
        
        try:
            print('Req')
            df = pd.read_csv(api_url2(params), usecols=cols)
            print('Records:', len(df))
            data.append(df)
        except Exception as e:
            print('Failed')
            
        time.sleep(10)
        
    filename = 'data/trade-value-{cc}-{years}.csv'.format(
        cc=params['cc'],
        years= '-'.join(map(lambda x: str(x), years))
    )
    print('Save to file:', filename)
    
    pd.concat(data).to_csv(
        filename,
        index=False
    )

https://comtrade.un.org/api/get?max=1000&type=C&freq=A&px=H1&ps=2013%2C2014%2C2015%2C2016%2C2017&r=660%2C28%2C32%2C533%2C44&p=all&rg=all&cc=270900&fmt=csv
Req
Records: 71
https://comtrade.un.org/api/get?max=1000&type=C&freq=A&px=H1&ps=2013%2C2014%2C2015%2C2016%2C2017&r=52%2C84%2C60%2C68%2C535&p=all&rg=all&cc=270900&fmt=csv
Req
Records: 80
https://comtrade.un.org/api/get?max=1000&type=C&freq=A&px=H1&ps=2013%2C2014%2C2015%2C2016%2C2017&r=76%2C92%2C124%2C136%2C152&p=all&rg=all&cc=270900&fmt=csv
Req
Records: 482
https://comtrade.un.org/api/get?max=1000&type=C&freq=A&px=H1&ps=2013%2C2014%2C2015%2C2016%2C2017&r=170%2C188%2C192%2C531%2C212&p=all&rg=all&cc=270900&fmt=csv
Req
Records: 200
https://comtrade.un.org/api/get?max=1000&type=C&freq=A&px=H1&ps=2013%2C2014%2C2015%2C2016%2C2017&r=214%2C218%2C222%2C238%2C254&p=all&rg=all&cc=270900&fmt=csv
Req
Records: 125
https://comtrade.un.org/api/get?max=1000&type=C&freq=A&px=H1&ps=2013%2C2014%2C2015%2C2016%2C2017&r=304%2C308%2C312%2C320%2C328&p=all&rg=

### MERGE ALL FILES IN ONE

In [21]:
oil_trade = pd.concat([
    pd.read_csv('data/trade-value-270900-2011.csv'),
    pd.read_csv('data/trade-value-270900-2012.csv'),
    pd.read_csv('data/trade-value-270900-2013-2014-2015-2016-2017.csv'),
    pd.read_csv('data/trade-value-270900-2018-2019-2020-2021.csv'),
])

oil_trade.dropna(axis=0, how='all', inplace=True)
oil_trade.reset_index(drop=True, inplace=True)

In [23]:
oil_trade.to_csv('data/americas-oil-trade-2011-2021.csv', index=False)